<b>Author:</b> Matthew Parzych<br>
<b>Email:</b> mattparzych1@gmail.com<br>
<b>Mobile:</b> 415.680.8678<br>
<b>Date:</b> 03/15/2024<br>
<p>
This is my take on the Waymark take home assignment.
<p>
I've got 2 primary goals at the start of this assignment:<br>
1) Accuracy (hopefully, both answers are correct), and<br>
2) <b>No use of any 'For Loops' in the code.</b><br>
Use of For Loops is not scalable as dataset sizes grow and<br>
is generally frowned upon :(

In [1]:
# Import the necessary libraries
import pandas as pd
# The boto3 library makes downloading files from AWS S3 very simple
import boto3

Import the datasets from Waymark's public S3 bucket

In [2]:
resource = boto3.resource('s3', aws_access_key_id='AKIA...............TXT',
            aws_secret_access_key='bWGKTChCrTEJU...............GP7VoAc')
waymark_s3_bucket = resource.Bucket('waymark-assignment')
waymark_s3_bucket.download_file('patient_id_month_year.csv', 
                                'patient_id_month_year.csv')
waymark_s3_bucket.download_file('outpatient_visits_file.csv', 
                                'outpatient_visits_file.csv')
print('Done loading S3 datasets')

Done loading S3 datasets


Load the 'patient_id_month_year.csv' file.<br>
Drop any empty columns and rows.<br>
Convert date strings to datetime formatted columns.<br>
Save a copy (per the assigment instructions) that I'll load again in Step 2.

In [3]:
col_defns = {
    'patient_id': str,
    'enrollment_start_date': str,
    'enrollment_end_date': str
}
df = pd.read_csv('patient_id_month_year.csv', names=col_defns, 
                 dtype=col_defns, skiprows=[0])
# No extraneous columns exist, but lots of empty rows (drop them)
df.dropna(axis=0, how='all', inplace=True)
# convert the string dates to datetime format
df['enrollment_start_date'] = pd.to_datetime(df['enrollment_start_date'],
                                             format='%m/%d/%y')
# Create an additional column (enrollment_end_date) - 
# the last day of the enrollment month
df['enrollment_end_date']=df['enrollment_start_date']+pd.offsets.MonthEnd(0)
# save a copy of the dataframe to the file name
#indicated in the instruction
# this file will be imported near the end
df.to_csv('patient_enrollment_span.csv', index=False)
df.head(5)

,patient_id,enrollment_start_date,enrollment_end_date
0,ID0001,2023-01-01,2023-01-31
1,ID0001,2023-02-01,2023-02-28
2,ID0001,2023-03-01,2023-03-31
3,ID0001,2023-06-01,2023-06-30
4,ID0001,2023-08-01,2023-08-31


In [4]:
print("ANSWER 1: " + str(len(df)) + " (rows of data)")
print(str(len(df) + 1) + " (rows, including header)")

ANSWER 1: 7309 (rows of data)
7310 (rows, including header)


In [5]:
df.shape # Sanity check

(7309, 3)

In [6]:
# check for any duplicates in the dataset.
# The length should be the same (if no duplicates) after the groupby.
n1_unique_combinations = df.groupby(['patient_id', 
                                     'enrollment_start_date',
                                     'enrollment_end_date']).size()
# Another data sanity check - 
# there should also be 7309 rows in the grouped dataframe
n1_unique_combinations

patient_id  enrollment_start_date  enrollment_end_date
ID0001      2023-01-01             2023-01-31             1
            2023-02-01             2023-02-28             1
            2023-03-01             2023-03-31             1
            2023-06-01             2023-06-30             1
            2023-08-01             2023-08-31             1
                                                         ..
ID1000      2023-02-01             2023-02-28             1
            2023-05-01             2023-05-31             1
            2023-07-01             2023-07-31             1
            2023-10-01             2023-10-31             1
            2023-11-01             2023-11-30             1
Length: 7309, dtype: int64

Build up a data structure where 'patient_id' is the key,<br>
and a list of lists of the associated enrollment date ranges.

In [7]:
# Apply a lambda function to each row of the passed in dataframe.
# Exclude the patient_id -- that's the key field.
def createEnrollmentDateRangeList(df):
    df = df.loc[:, df.columns != 'patient_id']
    return df.apply(lambda row: list(row), axis=1).to_list()
    
enrollmentsDF = df.groupby('patient_id').apply(createEnrollmentDateRangeList)
enrollmentsDF.head(5)

patient_id
ID0001    [[2023-01-01 00:00:00, 2023-01-31 00:00:00], [...
ID0002    [[2023-02-01 00:00:00, 2023-02-28 00:00:00], [...
ID0003    [[2023-03-01 00:00:00, 2023-03-31 00:00:00], [...
ID0004    [[2023-01-01 00:00:00, 2023-01-31 00:00:00], [...
ID0005    [[2023-01-01 00:00:00, 2023-01-31 00:00:00], [...
dtype: object

Read in the outpatient_visits_file - remove any extraneous columns and rows.<br>
Convert 'date' column values to date time format for accurate comparisons.<br>
Convert outpatient_visit_count from string to int.

In [8]:
# This is Step 2, Part 1 of the assignment
df2 = pd.read_csv('outpatient_visits_file.csv', dtype=str)
df2.dropna(axis=0, how='all', inplace=True)
df2.dropna(axis=1, how='all', inplace=True)
df2['date'] = pd.to_datetime(df2['date'], format='%m/%d/%y')
df2['outpatient_visit_count'] = df2['outpatient_visit_count'].astype(int)
df2.head(5)

,patient_id,date,outpatient_visit_count
0,ID0001,2023-01-03,2
1,ID0001,2023-01-08,2
2,ID0001,2023-01-09,2
3,ID0001,2023-01-15,2
4,ID0001,2023-01-21,4


Determine if there are any days (for a given patient_id) with multiple visits records.<br>
<b>*** IMPORTANT STEP ***</b> I want to consolidate them all here so they don't negatively affect the<br>
'ct_days_with_outpatient_visit' calculations done later.<br>
I do this by aggregating them and summing their 'outpatient_visit_count' values.

In [9]:
df2.shape

(15328, 3)

In [10]:
n2_unique_combinations = df2.groupby(['patient_id', 'date']).size()
n2_unique_combinations

patient_id  date      
ID0001      2023-01-03    1
            2023-01-08    1
            2023-01-09    1
            2023-01-15    1
            2023-01-21    1
                         ..
ID0980      2023-07-26    1
            2023-09-28    1
            2023-10-20    1
            2023-11-25    1
            2023-12-16    1
Length: 14877, dtype: int64

Turns out there are some duplicated days (for a given patient_id) - aggregate and sum those values.

In [11]:
df2 = df2.groupby(['patient_id', 'date'], as_index=False).agg(
    {'patient_id': 'first', 'date': 'first', 
     'outpatient_visit_count': 'sum'}
)
df2.shape

(14877, 3)

Now, with the duplicates patient_id::date values consolidated,<br>
we can safely build up a second data structure.<p>
This one also has patient_id as the key, but the associated list of lists<br>
contain the date and number of visits for that particular date.

In [12]:
# Apply a lambda function to each row of the passed in dataframe.
# Exclude the patient_id -- that's the key field.
def createVisitDateQuantityList(df):
    df = df.loc[:, df.columns != 'patient_id']
    return df.apply(lambda row: list(row), axis=1).to_list()

visitsDF = df2.groupby('patient_id').apply(createVisitDateQuantityList)
visitsDF.head(5)

patient_id
ID0001    [[2023-01-03 00:00:00, 2], [2023-01-08 00:00:0...
ID0002    [[2023-01-15 00:00:00, 1], [2023-02-08 00:00:0...
ID0003    [[2023-01-08 00:00:00, 3], [2023-03-25 00:00:0...
ID0004    [[2023-01-27 00:00:00, 10], [2023-02-06 00:00:...
ID0005    [[2023-01-17 00:00:00, 3], [2023-01-31 00:00:0...
dtype: object

The following 2 functions support Step 2 part 2.

In [13]:
# 'iterate' determines which, if any, enrollment
# date ranges contain the visit date.
# When there's a match, it should always be a single date range
# (but I return a list, just in case).
# The matching date range is returned as output from the filter.
def iterate(lst, indx, end, id, dtRanges):
    # Recursion completed? If so, return.
    if indx < 0 or end >= len(lst) or indx > end:
        return
    # Otherwise, pull date date and number of visits from the list (lst)
    visitDt = lst[indx][0]
    numVisits = lst[indx][1]
    # Use filter and a lambda function here - to avoid use of For Loops
    match = list(filter(lambda x: x[0] <= visitDt and
                        x[1] >= visitDt, dtRanges))
    if (match) and (len(match)==1): # if a match exists,... 
        # ... there should only be 1 date range that matches
        # Increase the number of ct_outpatient_visits
        # for the matching enrollment period. 
        finalDF.loc[(finalDF.patient_id == id) &
                    (finalDF.enrollment_start_date == match[0][0]),
                    'ct_outpatient_visits'] += numVisits
        # Increment the number of ct_days_with_outpatient_visit
        # for the matching enrollment period.
        # This would have caused a problem if we didn't aggregate (sum up)
        # duplicate patient_id::date earlier
        finalDF.loc[(finalDF.patient_id == id) &
                    (finalDF.enrollment_start_date == match[0][0]),
                    'ct_days_with_outpatient_visit'] += 1
    # Otherwise, the visit didn't coincide with any enrollment periods. 
    # Make a note to figure out how these erroneous records get created.
    iterate(lst, indx + 1, end, id, dtRanges)

# 'populateVisitCounts' extract the patient_id, then gets the associated
# enrollment date ranges and visits for that patient_id
def populateVisitCounts(df):
    idUnderTest = df.iloc[0]['patient_id']
    dateRanges = enrollmentsDF[idUnderTest]
    visits = visitsDF[idUnderTest]
    lstLen = len(visits) - 1 # count starts with 0
    # Use recursion here to avoid using a 'For Loop'
    # Identify which, if any, enrollment date ranges this visit is in...
    iterate(visits, 0, lstLen, idUnderTest, dateRanges)

This is Step 2 Part 2 of the assigment. We read in the csv file created in Step 1.<br>
<i>The assigment instructions wanted this - but it doesn't seem necessary.</i>

In [14]:
finalDF = pd.read_csv('patient_enrollment_span.csv',
                      names=col_defns,
                      dtype=col_defns,
                      skiprows=[0])
finalDF['enrollment_start_date'] = pd.to_datetime(
    finalDF['enrollment_start_date'])
finalDF['enrollment_end_date'] = pd.to_datetime(
    finalDF['enrollment_end_date'])
# Add the 2 additional columns calle for by the assignment
# instructions and initialize the values to zero.
finalDF['ct_outpatient_visits'] = 0
finalDF['ct_days_with_outpatient_visit'] = 0
finalDF.head(5)

,patient_id,enrollment_start_date,enrollment_end_date,ct_outpatient_visits,ct_days_with_outpatient_visit
0,ID0001,2023-01-01,2023-01-31,0,0
1,ID0001,2023-02-01,2023-02-28,0,0
2,ID0001,2023-03-01,2023-03-31,0,0
3,ID0001,2023-06-01,2023-06-30,0,0
4,ID0001,2023-08-01,2023-08-31,0,0


Now we have the data loaded, the data structures built and populated - it's time<br>
to do the calculations of ct_outpatient_visits and ct_days_with_outpatient_visit<br>
for all the patient_ids and their associated visit dates.

In [15]:
# using visitsDF because not all enrollment periods have visits
uuids = visitsDF.reset_index()
uuids.groupby('patient_id').apply(lambda x: populateVisitCounts(x))
finalDF.head(5)

,patient_id,enrollment_start_date,enrollment_end_date,ct_outpatient_visits,ct_days_with_outpatient_visit
0,ID0001,2023-01-01,2023-01-31,13,6
1,ID0001,2023-02-01,2023-02-28,0,0
2,ID0001,2023-03-01,2023-03-31,4,2
3,ID0001,2023-06-01,2023-06-30,6,2
4,ID0001,2023-08-01,2023-08-31,6,2


In [16]:
# Step 2, Part 3 -- save the results 
finalDF.to_csv('result.csv', index=False)

In [17]:
num_unique_values = pd.unique(finalDF["ct_days_with_outpatient_visit"])
#print(num_unique_values)
# print the number of unique values
print("ANSWER 2: There are " + str(len(num_unique_values)) +
      " distinct values of ct_days_with_outpatient_visits")
print("(across all patient_ids).")

ANSWER 2: There are 9 distinct values of ct_days_with_outpatient_visits
(across all patient_ids).
